# Dynamics: simulation and control
This notebook focuses on the simulation of polyarticulated systems, with final exercices where simple control laws have to be designed. The first part deals with collision detection using the hppfcl module of pinocchio. We then build a complete simulation engine for rigid unilateral contacts. 

In [ ]:
import gepetuto.magic

## Set up

We will use several models in this tutorial:
- a simple scene with 3 convex objects (buildSceneThreeBodies)
- a variation of this first scene with more objects and walls (buildScenePillsBox)
- a stack of cubes of various size (buildSceneCubes)
- a robot hand

In [ ]:
from tp4.scenes import buildSceneThreeBodies, buildScenePillsBox, buildSceneCubes, buildSceneRobotHand

We rely on the HPP-FCL module of Pinocchio, which compute collision between geometries.

In [ ]:
import hppfcl
import pinocchio as pin
import numpy as np
import time
from supaero2025.meshcat_viewer_wrapper import MeshcatVisualizer
from tp4 import compatibility
import matplotlib.pylab as plt

## A basic example of collisions

The goad of this section is to introduce a simple example of collision distances between bodies, and the underlying notions of witness points and segment and normal direction.

Let's build a simple scene with 3 objects and display their proximity.

In [ ]:
# %load tp4/generated/example_display_witness_build
# Build a scene
model,geom_model = buildSceneThreeBodies()
data = model.createData()
geom_data = geom_model.createData()

# Start meshcat
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model)


In [ ]:
q=pin.randomConfiguration(model)
viz.display(q)

In [ ]:
viz.viewer.jupyter_cell()

We can compute the distances between the 3 objects. Let's do it and show the closest points.

In [ ]:
from tp4.display_witness import DisplayCollisionWitnessesInMeshcat

In [ ]:
# %load tp4/generated/example_display_witness_witness
# Build the viewer add-on to display the witnesses.
mcWitnesses = DisplayCollisionWitnessesInMeshcat(viz)


In [ ]:
pin.computeDistances(model,data,geom_model,geom_data,q)
mcWitnesses.displayDistances(geom_data)

Each collision pair corresponds to a pair of closets points, respectively located at the surface of the collision geometries if they are not colliding. These points are sometime called the *witness* points.

The witness points are connected by the *witness* segment. This segment is normal to the two collision surfaces, in the case each surface is smooth around the witness point. The normalized direction is called the collision *normal*. Its orientation is a convention (the most logical convention is to go from body 1 to body 2 of the collision pair).

Let's move the objects to better see these witness elements.

In [ ]:
# %load tp4/generated/example_display_witness_trajectory
v = (np.random.rand(model.nv)*2-1)*1e-3
r0 = [ np.linalg.norm(q[7*i:7*i+3]) for i in range(model.nq//7) ]
for t in range(100):

    # Update the robot position along an arbitrary trajectory
    q = pin.integrate(model,q,v*10)
    for i in range(model.nq//7):
        q[7*i:7*i+3] *= r0[i]/np.linalg.norm(q[7*i:7*i+3])
    viz.display(q)

    # Display the witness points
    pin.computeDistances(model,data,geom_model,geom_data,q)
    mcWitnesses.displayDistances(geom_data)

    time.sleep(.01)


## Details about Pinocchio HPP-FCL 

Let's see now how to extract the contact and distance information from Pinocchio and the module HPP-FCL, and how to store this in a proper data structure.

### Geometry model
The geometry model contains a set of object, each described by a name, a geometry primitive and a placement with respect to a parent joint.

In [ ]:
geom1 = geom_model.geometryObjects[0]
geom1.name, geom1.parentJoint, geom1.geometry, geom1.placement

In addition, we also store the pairs of geometry objects that should be considered when evaluating collisions and distances.

In [ ]:
list(geom_model.collisionPairs)

### HPP-FCL computeDistances and computeCollisions

The geometry algorithms are implemented in HPP-FCL under two different sets of functions, which respectively compute the distance between bodies, and the collision between bodies.
When computing the distance, a unique pair of witness points is produced, and the signed distance is also evaluated.
When computing the collision, an effort is made to compute all the contact points, and early stop can be activated to reduce the algorithm cost as soon as a collision is found. 
All in all, for this initiation, both can be considered quite similarly.

Both functions are parametrized by a *request* object, and write their output in a *result* object. *Request* and *result* objects are preallocated in the geometry data, one of each for each pair of collisions. If you activate or deactivate a collision pair, you have to regenerate these objects (and so if you add a new geometry object in the list). 

In [ ]:
len(geom_model.collisionPairs), len(geom_data.collisionRequests), len(geom_data.distanceResults)

The placement of the geometry objects with respect to world frame are stored in geom_data.oMg, and computed as
$$^oM_g(q) = ^oM_i(q) ^iM_g$$
with $^oM_g$ the placement of the object wrt world, $^oM_i$ the placement of the parent joint and $^iM_g$ the (fixed) placement of the object wrt the parent joint.

In [ ]:
len(geom_model.geometryObjects),len(geom_data.oMg)

This computation is triggered by *pin.updateGeometryPlacements*, after forward kinematics has been run or by forcing the refresh of the forward kinematics.

In [ ]:
pin.updateGeometryPlacements(model,data,geom_model,geom_data,q)

The computation of the distances and collisions is triggered by their respective function and by default forces the kinematic update.

In [ ]:
pin.computeCollisions(model,data,geom_model,geom_data,q)
pin.computeDistances(model,data,geom_model,geom_data,q)

These two functions actually iterate over each pair of collisions and run *pin.computeDistance* or *pin.computeCollision*, which in turn runs *hppfcl.distance* and *hppfcl.collide*. 
HPP-FCL directly works with geometry placements, ignoring the kinematics and the configuration space which is provided by Pinocchio.
All these functions have very similar signatures, and Pinocchio is mostly doing a basic wrapping around the HPP-FCL library and gently connects the forward kinematics with the collision algorithms. 

### The *request* objects
For collisions, Pinocchio mostly forward the requests to the library HPP-FCL, recently renamed COAL. It is possible to parametrize the algorithm computing the collision by setting the *request* objects stored in the geometry data, one for each collision pair:

In [ ]:
for req in geom_data.collisionRequests:
    req.security_margin = 1e-3 # Collision is detected with a millimeter margin
    req.num_max_contacts = 5 # Max number of collision point
    req.enable_contact = True # To force the computation of the normal


The same is true for the *distance* algorithm.

In [ ]:
for req in geom_data.distanceRequests:
    req.security_margin = 1e-3 # Collision is detected with a millimeter margin
    req.num_max_contacts = 5 # Max number of collision point
    req.enable_contact = True # To force the computation of the normal

Then, re-evaluate the collisions.

In [ ]:
pin.computeCollisions(model,data,geom_model,geom_data,q)
pin.computeDistances(model,data,geom_model,geom_data,q)

### The *result* objects
The distance result contains the pair of witness points $p_1$ and $p_2$, the normal direction pointing from $p_1$ to $p_2$ and the signed distance.

In [ ]:
d = geom_data.distanceResults[0]
print("p1:",d.getNearestPoint1())
print("p2:",d.getNearestPoint2())
print("n:",d.normal)
print("dist:",d.min_distance)
print("check:",np.cross(d.normal,d.getNearestPoint2()-d.getNearestPoint1())) # The two vectors are parallel.

The collision object contains similar information, but can stores several witness pairs instead of a single one, or none if there is no collisions.
The collision is decided based on a security margin, tuned in the collision request object.

In [ ]:
print('Margin:',geom_data.collisionRequests[0].security_margin)
c = geom_data.collisionResults[0]
print('Number of collisions:',len(c.getContacts()))

Now choose a configuration in collision and look at the content of this contact list.

In [ ]:
%do_not_load_snippet tp4/generated/example_find_collisions_find

In [ ]:
# %load tp4/generated/example_find_collisions_print
for pairId,c in enumerate(geom_data.collisionResults):
    if len(c.getContacts())>0:
        contact = c.getContact(0)
        print([ n for n in dir(contact) if '__' not in n])
        break


## From collision detection to contact model

### The maths
Once the collision are detected, we have to decide the model of the contact interaction that will happen at this location. It can range from a fixed 3D contact (bilateral), a sliding contact enforcing only constraints along the normal direction, a frictional contact or even a full 6D contact with no degree of freedom between the two corresponding bodies (and many other models you can imagine).
To describe the contact model, we best formulate it in a reference frame at the contact location. Here we arbitrarily decide to align the *z* direction of the contact frame with the normal direction (oriented from body 1 to body 2) and set the two other *x* and *y* directions to any arbitrary orthogonal basis of the tangential plane. If the collision distance is exactly 0 (which will never be), the center of the frame should be at the contact point. In practice, we can define either two contact frames at each witness point or a single one at the middle point between both. Since the contact distance will never be large, it should be approximately the same and both solutions are acceptable in practice.

In [ ]:
OC1 = contact.getNearestPoint1()
OC2 = contact.getNearestPoint2()
normal = contact.normal

From these vectors, build the two contact frames $^oM_{c1}$ and $^oM_{c2}$ centered in $c_1$  and $c_2$ and with *z* axis aligned with the *normal* direction. You can for example use the pin.Quaternion.FromTwoVectors(v1,v2) function, which returns a rotation $r(.)$ such that $r(v_1)=v_2$ ie that transforms $v_1$ into $v_2$ (here we want to transform the *z* axis $z=[0,0,1]$ into the *normal* vector).

In [ ]:
# %load tp4/generated/create_rigid_contact_models_for_hppfcl_frames
pair = geom_model.collisionPairs[pairId]
gid1, gid2 = pair.first, pair.second
g1 = geom_model.geometryObjects[gid1]
g2 = geom_model.geometryObjects[gid2]
jid1 = g1.parentJoint
jid2 = g2.parentJoint
oMj1 = data.oMi[jid1]
oMj2 = data.oMi[jid2]

# Compute translation and rotation of the contact placements
# If dist=0, both placements are identical (and should be somehow close
# when dist is reasonibly small).
quat = pin.Quaternion.FromTwoVectors(
    pin.ZAxis, normal
)  # orientation of the contact frame wrt world
assert np.isclose(quat.norm(), 1)
oMc1 = pin.SE3(quat.matrix(), OC1)  # Placement of first contact frame in world
oMc2 = pin.SE3(quat.matrix(), OC2)  # Placement of second contact frame in world


The pin.RigidConstaintModel class in Pinocchio conveniently stores these contact quantities: the two parent joints that carries the corresponding colliding bodies, the placements of the contact frames with respect to the joint frames $^{j1}M_{c1}$, $^{jc}M_{c2}$, and a flag specifying if the contact model is 3D (point contact) or 6D (surface contact), which will later be convenient to evaluate the corresponding Jacobian.

In [ ]:
# %load tp4/generated/create_rigid_contact_models_for_hppfcl_model
contact_model = pin.RigidConstraintModel(
        pin.ContactType.CONTACT_3D,
        model,
        jid1,oMj1.inverse()*oMc1,
        jid2,oMj2.inverse()*oMc2,
        pin.LOCAL)


In [ ]:
contact_data = contact_model.createData()

### Creation wrapper
This code has been conveniently wrapped in a dedicated function, that takes all the collision or distance results and returns a list of contact models.

In [ ]:
from tp4.create_rigid_contact_models_for_hppfcl import createContactModelsFromCollisions,createContactModelsFromDistances

In [ ]:
# %load tp4/generated/create_rigid_contact_models_for_hppfcl_example
pin.computeCollisions(model,data,geom_model,geom_data,q,False)
contact_models = createContactModelsFromCollisions(model,data,geom_model,geom_data)
contact_datas = [ cm.createData() for cm in contact_models ]

pin.computeDistances(model,data,geom_model,geom_data,q)
contact_models = createContactModelsFromDistances(model,data,geom_model,geom_data,
                                                       threshold=10) # threshold in meter
contact_datas = [ cm.createData() for cm in contact_models ]


### Display the contact models
A robot description (like the robot URDF) typically describes two sets of geometries: a first set, accurate, detailled and texture fore visualization and non real-time evaluations (called the *visual* model); a second one, less detailed, sometimes approximated or convexified, for efficient computations (often called the *collision* model).

Let's display in the 3D viewer the contact models with red contact patches (disks) located at the center of the contact frames and aligned with the *x-y* contact plane. 
To make it easy to render, we will put these patches in the *visual* model of the system. 
A fix set of patches is preallocated at initialization (and hidden or displaced far from the scene center when not useful). When contacts are created, the patches are revealed and properly place for rendering.
The functions in the file display_collision_patches have be written for that.

In [ ]:
from tp4.display_collision_patches import preallocateVisualObjects,updateVisualObjects

In [ ]:
# %load tp4/generated/example_display_collision_patches_create
# Obtained by simply copying the collision model
visual_model = geom_model.copy()
preallocateVisualObjects(visual_model)

# Start meshcat
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=visual_model)


In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
# %load tp4/generated/example_display_collision_patches_display
updateVisualObjects(model,data,contact_models,contact_datas,visual_model,viz)
viz.display(q)


You have to run updateVisualObjects each time you change q, and before calling viz.display(q).

##  Building a (very) simple kinematic simulator

Let's build a very simple collision simulator. Starting from an initial configuration where some bodies are colliding, you have to choose a velocity of each body to disentangle the colliding parts.
The scene created above is composed of 3 bodies, each attached to a "free-flyer" joint. The configuration of the scene is the concatenation of the configuration of each joint, corresponding to translation and quaternion describing the placement of each body. The configuration velocity also is the concatenation of the velocity of each body, each corresponding to the spatial velocity $\nu = (v,\omega)$ of each body expressed in the local frame. 

In [ ]:
# %load tp4/generated/example_separate_objects_hyperparams
# HYPER PARAMETERS OF THE PUSH STRATEGY
PUSH_FACTOR = .1
EPSILON = 1e-1
NB_ITER = 100
# Compute the contact information based on distances or collisions?
USE_DISTANCE = True


At each iteration, you have to choose a configuration velocity, then integrate it.

In [ ]:
# Set minimal distance to be EPSILON
# (can be zero, but the rendering is less clear).
for r in geom_data.collisionRequests:
    r.security_margin = EPSILON

# Keep distance history for active pairs (indexed by contact name)
h_dist = {}

# Iteratively push the colliding pairs ...
for i in range(NB_ITER):

    # We will compute a change of configuration dq.
    # 0 if no active pair.
    dq = np.zeros(model.nv)

    # Compute the collision at current configuration.
    if USE_DISTANCE:
        pin.computeDistances(model,data,geom_model,geom_data,q)
    else:
        pin.computeCollisions(model,data,geom_model,geom_data,q)

    # From hppfcl contact information, build a pin.RigidContactModel
    if USE_DISTANCE:
        contact_models = createContactModelsFromDistances(model,data,geom_model,geom_data,EPSILON)
    else:
        contact_models = createContactModelsFromCollisions(model,data,geom_model,geom_data)
    contact_datas = [ cm.createData() for cm in contact_models ]

    # For each detected contact ...
    for cmodel,cdata in zip(contact_models,contact_datas):

        # Recover contact information
        jid1 = cmodel.joint1_id
        j1Mc1 = cmodel.joint1_placement
        jid2 = cmodel.joint2_id
        j2Mc2 = cmodel.joint2_placement

        # Compute signed distance
        oMc1 = cdata.oMc1 = data.oMi[jid1]*j1Mc1
        oMc2 = cdata.oMc2 = data.oMi[jid2]*j2Mc2
        dist = oMc1.actInv(oMc2.translation)[2]-EPSILON  # signed distance
        
        # ### TODO
        # Here, do something to the velocity of the two joints carrying the 
        # two colliding bodies.
        # Displacement for body 1
        dq[model.idx_vs[jid1]:model.idx_vs[jid1]+6] += 0 ### TODO FIX ME
        # Displacement for body 2
        dq[model.idx_vs[jid2]:model.idx_vs[jid2]+6] -= 0 ### TODO FIX ME
        
        # Log the distance in h_dist for future plot
        if cmodel.name not in h_dist:
            h_dist[cmodel.name] = np.zeros(NB_ITER)
        h_dist[cmodel.name][i] = dist

    # Finally, modify the current config q with the push dq
    q = pin.integrate(model,q,dq)

    # Display the current configuration
    if i % 10 == 0:
        # Meshcat is slow to display the patches, display once in a while
        updateVisualObjects(model,data,contact_models,contact_datas,visual_model,viz)
        viz.display(q)


If you don't have any strong ideas, you can start to compute a "push" repulsive 3D velocity that the first witness point should adopt. The second witness point should likely adopt the opposite velocity. This is the value of the velocity vector field at the contact point. Now choose the spatial velocity at the center of the body so that the value of the corresponding vector field matches. In a first trial, a spatial velocity with 0 angular velocity $\omega$ should be evident to guess. Can you imagine a more efficient linear+angular velocity that would work as well?

In [ ]:
%do_not_load_snippet tp4/generated/example_separate_objects_loop

### With a more complex scene
Now let's load a square jar full of pills. Run the same algorithm on it. 

In [ ]:
# %load tp4/generated/scenes_pills
model,geom_model = buildScenePillsBox(seed=2,nobj=30,wall_size=2.0,one_of_each=True)
visual_model = geom_model.copy()
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model)

# Generate colliding configuration
data = model.createData()
geom_data = geom_model.createData()
for i in range(10):
    q0 = pin.randomConfiguration(model)
    pin.computeCollisions(model,data,geom_model,geom_data,q0)
    if sum([ len(c.getContacts()) for c in geom_data.collisionResults ])>10:
        break
    print(sum([ len(c.getContacts()) for c in geom_data.collisionResults ]))


In [ ]:
q = q0.copy()
viz.display(q)
viz.viewer.jupyter_cell()

You can plot the convergence of the colliding bodies from initial negative (penetration) distance to 0.

In [ ]:
# %load tp4/generated/example_separate_objects_plot
# Plot the distances
for k,v in h_dist.items():
    h = plt.plot(v,label=k)
plt.legend()


## Contact-less simulator

We now have all the contact information, let's look at the system dynamics. 
We will write first a contact-less simulator, just integrating free fall.
Better use an actuated model for that.

### Model of a robot hand

In [ ]:
model,geom_model = buildSceneRobotHand()
q0 = model.referenceConfigurations['default']
data = model.createData()

In [ ]:
visual_model = geom_model.copy()
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model)
viz.display(q0)
viz.viewer.jupyter_cell()

### Evaluating the core elements of the dynamics

In a first time, let's play with the dynamics without constraints.
Choosing an arbitrary joint torque $\tau_q$, inocchio can compute the robot acceleration and integrate it.
The dynamic equation of the robot is $M a_q + b = \tau_q$, with $M$ the mass, $a_q$ the joint acceleration and $b$ the drift.
The mass matrix can be computed using *CRB* algorithm (function of q). The drift is computed using *NLE* (nonlinear effects) algorithm (function of $q$, $v_q$). 


In [ ]:
# %load tp4/generated/example_control_init
q = q0.copy()
vq = np.zeros(model.nv)


In [ ]:
# %load tp4/generated/example_control_mass
M = pin.crba(model, data, q)
b = pin.nle(model, data, q, vq)


These terms correspond to the inverse dynamics. They can be numerically inverted to compute the direct dynamics.

Using $M$ and $b$ computed by the above algorithms, and knowing a given set of joint torques $\tau_q$, how would you compute $a_q$ so that $M a_q + b = \tau_q$?

In [ ]:
%do_not_load_snippet tp4/generated/example_control_dyninv

In [ ]:
# %load tp4/generated/example_control_dyninv
tauq = np.random.rand(model.nv)*2-1
aq = np.linalg.inv(M) @ (tauq - b)


The inverse-dynamics algorithm indeed compute the needed torques to achieve a given acceleration. We can use the *RNEA* function to double-check our result:

In [ ]:
print(pin.rnea(model,data,q,vq,aq)-tauq)
print(pin.rnea(model,data,q,vq,vq*0)-pin.nle(model,data,q,vq))

### Integrating the acceleration

Once aq as been computed, it is straight forward to integrate it to velocity using $v_q += a_q \Delta t$. 
Integration to joint position is more complex in general, as we saw for the mobile robot. Here, $n_q = n_v$ and everything is Euclinea, so a simple += integration would work, but since it is implemented in pinocchio, let's keep the good habits:

In [ ]:
# %load tp4/generated/example_control_hyper
# Hyperparameters for the simu
DT = 1e-3  # simulation timestep
DT_VISU = 1/50
DURATION = 3. # duration of simulation
T = int(DURATION/DT) # number of time steps


In [ ]:
# %load tp4/generated/example_control_integrate
vq += aq * DT
q = pin.integrate(model, q, vq * DT)


Now, you should be able to implement a first simulation (not a rendering any more) of the robot behavior when no torque is applied (tauq = 0).



In [ ]:
tauq = np.zeros(model.nv)

Fill the template below to get a free-falling dynamics.

In [ ]:
for i in range(T):
    t = i * DT

    # Compute the model.
    # TODO FIX ME

    # Simulated the resulting acceleration (forward dynamics
    # TODO FIX ME
    
    # Integrate the acceleration.
    # TODO FIX ME
    
    # Display once in a while...
    if DT_VISU is not None and abs((t) % DT_VISU)<=0.9*DT:
        viz.display(q)
        time.sleep(DT_VISU)

Now modify the simulator to encompass joint friction, i.e. the torque is opposite to the velocity with a friction coefficient $K_f$ (take $K_f=0.1$ as a start).

### (optional) Reference trajectory
If you like it, a class is implemented to compute a reference trajectory as a sinus with various frequency, phase and amplitude on each joint. It is implemented in traj_ref with the class TrajRef.

In [ ]:
from tp4.traj_ref import TrajRef

In this simple example, we define a trajectory in dimension 3, starting at t=0 from q=[0,0,0], with 3 sinusoid of frequency 1,2 and 3, all with the same amplitude 1.5.

In [ ]:
# %load  tp4/generated/traj_ref_main
qdes = TrajRef(
    np.array([0, 0, 0.0]), omega=np.array([1, 2, 3.0]), amplitude=1.5
)
t = 0.2
print(qdes(t), qdes.velocity(t), qdes.acceleration(t))


### Proportional-derivative and computed torque

Now choose a reference joint position (possibly time varying, like in the hand example).
The joint torques can then be computed to track the desired position, with $\tau_q = -K_p (q-q^{des}) - K_v v_q$. Both gains $K_p$ and $K_v$ should be properly chosen. Optimal tracking is obtained with $K_v = 2 \sqrt{K_p}$. 
In general, a desired velocity is also tracked to avoid tracking errors.



First choose the PD gains:

In [ ]:
# %load tp4/generated/example_control_hyper_control
# Hyperparameters for the control
Kp = 50.0  # proportional gain (P of PD)
Kv = 2 * np.sqrt(Kp)  # derivative gain (D of PD)


Then choose the reference trajectory.

In [ ]:
# %load tp4/generated/example_control_trajref
from tp4.traj_ref import TrajRef  # noqa E402

qdes = TrajRef(
    q0,
    omega=np.array([0, 0.1, 1, 1.5, 2.5, -1, -1.5, -2.5, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]),
    amplitude=1.5,
)


Finally, implement the control loop using the following template.

In [ ]:
hq    = []   ### For storing the logs of measured trajectory q
hqdes = []   ### For storing the logs of desired trajectory qdes
for i in range(10000):
    t = i*DT

    # Compute the PD control.
    tauq = np.zeros(model.nv)   # REPLACE ME
    
    # Compute the model M and b.
    M = np.eye(model.nv)        # REPLACE ME
    b = np.zeros(model.nv)      # REPLACE ME
    
    # Simulated the resulting acceleration (forward dynamics
    aq = np.zeros(model.nv)     # REPLACE ME

    # Integrate the acceleration.
    vq += np.zeros(model.nv)    # REPLACE ME
    q  = q+np.zeros(model.nv)   # REPLACE ME

    # Display once in a while...
    if DT_VISU is not None and abs((t) % DT_VISU)<=0.9*DT:
        viz.display(q)
        time.sleep(DT_VISU)

    # Log the history.
    hq.append(q.copy())
    hqdes.append(qdes.copy())



Here is the solution, should you need it.

In [ ]:
%do_not_load_snippet tp4/generated/example_control_loop

Here, there is a strong coupling between joints, due to the mass matrix that is not compensated in the simple PD law. In theory, the computed torques is to compute the joint torque by inverse dynamics from a reference joint acceleration. This boils down to canceling the simulation equation by choosing the proper terms in the control law. It is now very interesting to implement in case of perfect dynamics knowledge. It might be more interesting to study in case the simulation is done with the perfect M, while the control is computed with approximate M (for example, using only the diagonal terms of the mass matrix). Let's rather simulate contact.


It is intersting to plot the results. In the solution of the previous example, the effective and desired position q and qdes have been stored in a log list. We can display them with matplotlib.

In [ ]:
plt.subplot(211)
plt.plot([q[2] for q in hq])
plt.plot([q[2] for q in hqdes])
plt.ylabel('Joint #2')
plt.subplot(212)
plt.plot([q[3] for q in hq])
plt.plot([q[3] for q in hqdes]);
plt.ylabel('Joint #1')
plt.xlabel('Iterations')

## Friction-less simulation

We will now combine the two aspects of the model: the collisions between geometries, and the dynamics of inertias. 

### Choose a model 
Let's come back to a simple model first: a cube on a floor.

In [ ]:
# %load tp4/generated/example_simu_frictionless_init
# ### SCENE
# Create scene with multiple objects
model,geom_model = buildSceneCubes(3)

# Create the corresponding data to the models
data = model.createData()
geom_data = geom_model.createData()

for req in geom_data.collisionRequests:
    req.security_margin = 1e-3
    req.num_max_contacts = 20
    req.enable_contact = True

# ### VIZUALIZATION
visual_model = geom_model.copy()
preallocateVisualObjects(visual_model,10)
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=visual_model)
updateVisualObjects(model,data,[],[],visual_model,viz)

# ### INIT MODEL STATE
q0 = model.referenceConfigurations['default']

viz.display(q0)


In [ ]:
viz.viewer.jupyter_cell()

### Contact Jacobian

The pin.RigidConstraintModel makes it possible to directly compute the corresponding jacobian. When considering 3D contact models, the jacobian maps the configuration velocity to the relative velocity of the two contact points in the local frame attached to the first witness.
Let's build a basic contact model to show the API:

In [ ]:
contact_model = pin.RigidConstraintModel(pin.ContactType.CONTACT_3D,model,1,pin.SE3.Identity(), 0, pin.SE3.Identity(), pin.LOCAL)
contact_data = contact_model.createData()

In [ ]:
pin.computeJointJacobians(model,data,q0)
pin.getConstraintsJacobian(model,data,[contact_model],[contact_data])

The constraint Jacobian express the constraint in term of velocity of body 1 wrt to body 2. It is more intuitive to think to the opposite: so you have to take the negative Jacobian (ie velocity of body 2 wrt body 1, whose normal component should be positive). Here, we said we will only consider the normal component of the contact, so select the last row (and if there is more than one contact point, every next rows over 3).

In [ ]:
Jc = -pin.getConstraintsJacobian(model,data,[contact_model],[contact_data])[2::3]

### Simulation without friction: primal formulation

You are now in position to write your first complete (friction-less) collision simulator.
The state of the system is the configuration $q$ and velocity $v_q$. The joint torques have to be decided (by a controller) as a function of the state. Here we take them constant null as the cube is unactuated.

In [ ]:
q = q0.copy()
v = np.zeros(model.nv)
tau = np.zeros(model.nv)

At each new simulation cycle, you need to choose the next velocity. 
If there is no contact, then the velocity is simply obtained after one step of free fall:

In [ ]:
vf = v + DT * pin.aba(model, data, q, v, tau)

If there is some collisions, then the next velocity will be as close as possible to the free fall ("as close as possible" in the sense of the metrics imposed by the inertia matrix), while respecting the constraints of positive normal velocities. Mathematically, this writes:

$$\min_{v_+} f(v_+) = \frac{1}{2} \| v_+ - v_f \|_M^2 $$
such that
$$J_c v_+ \ge 0$$

where $f(v_+) = \frac{1}{2} \| v_+ - v_f \|_M^2 = = \frac{1}{2} (v_+-v_f)^T M (v_+-v_f) = \frac{1}{2} v_+^T M v_+ - (M v_f)^T v_+$

This defines a well-shaped LCPQ that can be solved by any off-the-shelf QP solver, [for example ProxQP, as explained in this companion notebook](appendix-proxqp.ipynb) .

Now write the simulation loop. At each simulation cycle:
1. Check the collisions, create the contact model and the corresponding contact data.
2. Evaluate the free-fall next velocity $v_f$ with ABA
3. If no collision, then the next velocity is v_f, continue.
4. If collisions, then evaluate the inertia matrix $M$ with pin.crba, and the Jacobian $J_c$ with pin.getConstraintJacobian; form the primal QP, solve it. The result is the next velocity, the Lagrange multipliers are the contact forces.
5. Optionally, print, display, log.

Here is a canvas:

In [ ]:
# %load tp4/generated/example_simu_frictionless_loop
# ### MAIN LOOP
# ### MAIN LOOP
# ### MAIN LOOP
for t in range(T):

    # Compute free dynamics
    tau = np.zeros(model.nv)
    pin.computeCollisions(model, data, geom_model, geom_data, q)
    vf = v + DT * pin.aba(model, data, q, v, tau)

    # Create contact models from collision
    contact_models = createContactModelsFromCollisions(model,data,geom_model,geom_data)
    contact_datas = [ cm.createData() for cm in contact_models ]
    
    nc = len(contact_models)
    if nc==0:
        # No collision, just integrate the free dynamics
        v = vf
    else:
        # With at least one collision ...
        # Compute mass matrix.
        # (pin.crba should be enough in P3X, but CAT to be sure in P2X);
        pin.computeAllTerms(model, data, q, v)
        # The contact solver express the constraint in term of velocity of
        # body 1 wrt to body 2. It is more intuitive to think to the opposite
        # so take the negative jacobian (ie velocity of body 2 wrt body 1, whose
        # normal component should be positive).
        # Select only normal components of contact
        # (frictionless slide on the tangent components, uncontrained)
        J = -pin.getConstraintsJacobian(model, data, contact_models, contact_datas)[2::3,:]
        assert(J.shape == (nc,model.nv))

        M = data.M

        # ## TODO: formulate the LCQP to compute vnext
        v = np.zeros(model.nv)
    
    # Finally, integrate the valocity
    q = pin.integrate(model , q, v*DT)

    # Visualize once in a while
    if DT_VISU is not None and abs((t*DT) % DT_VISU)<=0.9*DT:
        updateVisualObjects(model,data,contact_models,contact_datas,visual_model,viz)
        viz.display(q)
        time.sleep(DT_VISU)

Should you need it, here is the primal formulation:

In [ ]:
%do_not_load_snippet tp4/generated/example_simu_frictionless_primal

### Delassus matrix

The Dual formulation requires another matrix: the Delassus operator, defined as:
$$\Lambda_c = J_c M^{-1} J_c^T$$
Physically, this is the inverse of the apparent (or operational, or contact) inertia, i.e. the mass that an operator would feel if it touches the system in this configuration at the contact points.
The pin.RigidConstraintModel makes it possible to evaluate $\Lambda_c$ more efficiently, but for now that's enough for us.

In [ ]:
Minv = pin.computeMinverse(model,data,q)
delasus = J@Minv@J.T

### (optional) Simulation without friction: primal formulation

The dual of the contact LCQP that we wrote is:
$$\min_\lambda g(\lambda) = \frac{1}{2} \lambda^T \Lambda_c \lambda + (J_c v_f)^T \lambda$$
subject to
$$\lambda \ge 0$$

Here, the decision variable is $\lambda$ which can be understood as the contact impulses (i.e. $\lambda = f.DT$, the effect of the forces during the timestep). The joint velocity is not appearing in this problem, as all quantities are in the operation (contact) space. 

Once the contact forces are found, the configuration velocity can be evaluated by calling the pin.aba algorithm, while specifying the contact forces.

In [ ]:
forces = np.zeros(Jc.shape[0])
vnext = v + DT * pin.aba(model, data, q, v, tau + J.T @ forces/DT)

Now rewrite the simulation problem with the dual formulation. Here is the canvas:

In [ ]:
# %load tp4/generated/example_simu_frictionless_loop
# ### MAIN LOOP
# ### MAIN LOOP
# ### MAIN LOOP
for t in range(T):

    # Compute free dynamics
    tau = np.zeros(model.nv)
    pin.computeCollisions(model, data, geom_model, geom_data, q)
    vf = v + DT * pin.aba(model, data, q, v, tau)

    # Create contact models from collision
    contact_models = createContactModelsFromCollisions(model,data,geom_model,geom_data)
    contact_datas = [ cm.createData() for cm in contact_models ]
    
    nc = len(contact_models)
    if nc==0:
        # No collision, just integrate the free dynamics
        v = vf
    else:
        # With at least one collision ...
        # Compute mass matrix.
        # (pin.crba should be enough in P3X, but CAT to be sure in P2X);
        pin.computeAllTerms(model, data, q, v)
        # The contact solver express the constraint in term of velocity of
        # body 1 wrt to body 2. It is more intuitive to think to the opposite
        # so take the negative jacobian (ie velocity of body 2 wrt body 1, whose
        # normal component should be positive).
        # Select only normal components of contact
        # (frictionless slide on the tangent components, uncontrained)
        J = -pin.getConstraintsJacobian(model, data, contact_models, contact_datas)[2::3,:]
        assert(J.shape == (nc,model.nv))

        Minv = pin.computeMinverse(model,data,q)
        delasus = J@Minv@J.T

        # ## TODO: formulate the dual LCQP to compute lambda
        forces = np.zeros(delasus.shape[0])

        # The next velocity results from the contact forces
        vnext = v + DT * pin.aba(model, data, q, v, tau + J.T @ forces/DT)

    # Finally, integrate the valocity
    q = pin.integrate(model , q, v*DT)

    # Visualize once in a while
    if DT_VISU is not None and abs((t*DT) % DT_VISU)<=0.9*DT:
        updateVisualObjects(model,data,contact_models,contact_datas,visual_model,viz)
        viz.display(q)
        time.sleep(DT_VISU)

Here is the dual formulation, should you need it.

In [ ]:
%do_not_load_snippet tp4/generated/example_simu_frictionless_dual

### Comparison: primal / dual

Once you have implemented both primal and dual formulations, you can double check that the results are the same.

In [ ]:
%do_not_load_snippet tp4/generated/example_simu_frictionless_check

## Salade de fruit

Let's now put all that together and make something nice out of it. You will implement a simple grasping strategy for the robot hand to hold an cube, then implement a PD controller and validate it in the simulator to check if your grasp is stable.

1. Load the hand model with an object.

In [ ]:
model,geom_model = buildSceneRobotHand(with_item=True)

2. For an arbitrary placement of the object that you will select, and a random initial configuration of the hand, progressively close the hand while monitoring the collision. Once a collision is created, either block the corresponding finger or (better) project the velocity in the null space of the constraint Jacobian to further maintain the contact.
3. Stop when you have enough contact points.
4. Search the minimal joint torque and corresponding contact forces such that the dynamics is enforced and the forces are sufficiently positive:



$$\min_{\tau,f} \| \tau \|^2$$
such that
$$g = \begin{bmatrix} 0 \\ \tau \end{bmatrix} + J_c^T f$$
$$f \ge \epsilon$$
with $g=g(q)$ the vector of joint gravity and $\epsilon$ a robustness threshold for the forces (take care that the object is unactuacted).

5. Put hand and object in your simulator, set a PD controller with the reference configuration you like, and check that your object stay static. It is acceptable that it falls (we cannot win all the time).

**Implement this solution in a standalone python script or notebook which 1. choose a grasp configuration, 2. compute a pair of grasp torques and forces and 3. simulate the grasp with a frictionless model.**